# Heart Attack Prediction


## Pemahaman Data

- **Age**: Usia pasien
- **Sex**: Jenis kelamin pasien
- **exang**: Angina yang diinduksi oleh olahraga (1 = ya; 0 = tidak)
- **ca**: Jumlah pembuluh darah utama (0-3)
- **cp**: Jenis nyeri dada
    - Nilai 1: Angina tipikal
    - Nilai 2: Angina atipikal
    - Nilai 3: Nyeri non-angina
    - Nilai 4: Asimtomatik
- **trtbps**: Tekanan darah istirahat (dalam mm Hg)
- **chol**: Kolesterol dalam mg/dl yang diambil melalui sensor BMI
- **fbs**: (gula darah puasa > 120 mg/dl) (1 = benar; 0 = salah)
- **rest_ecg**: Hasil elektrokardiografi istirahat
    - Nilai 0: Normal
    - Nilai 1: Memiliki abnormalitas gelombang ST-T (Inversi gelombang T dan/atau elevasi atau depresi ST > 0.05 mV)
    - Nilai 2: Menunjukkan hipertrofi ventrikel kiri yang mungkin atau pasti menurut kriteria Estes
- **thalach**: Denyut jantung maksimum yang dicapai
- **target**: 0 = peluang serangan jantung lebih rendah, 1 = peluang serangan jantung lebih tinggi

<a href='https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?select=heart.csv'>Link Dataset<a/>

## Menyiapkan Pustaka

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier


from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler

## Gathering Data

In [43]:
df = pd.read_csv('/content/heart.csv')

In [44]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [45]:
baskom = df.shape
print('Jumlah Baris :',baskom[0])
print('Jumlah Kolom :',baskom[1])

Jumlah Baris : 303
Jumlah Kolom : 14


## Assesing Data

In [46]:
df.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [48]:
df.isna().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [49]:
print('Jumlah Data Duplikat :',df.duplicated().sum())

Jumlah Data Duplikat : 1


In [50]:
df = df.drop_duplicates()
print('Jumlah Data Duplikat :',df.duplicated().sum())

Jumlah Data Duplikat : 0


In [51]:
x = df.drop(columns=['output'],axis=1)
y = df['output']

In [52]:
# Membagi dataset menjadi data latih & data uji
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [53]:
X_train.shape

(241, 13)

In [54]:
X_test.shape

(61, 13)

## Modeling

### KNN

In [62]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
knn_predictions = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("K-Nearest Neighbors Accuracy: {:.2%}".format(knn_accuracy))

K-Nearest Neighbors Accuracy: 68.85%


### Decision Tree

In [60]:
tree = DecisionTreeClassifier(random_state=1)
tree.fit(X_train, y_train)
tree_predictions = tree.predict(X_test)
tree_accuracy = accuracy_score(y_test, tree_predictions)
print("Decision Tree Accuracy : {:.2%}:".format(tree_accuracy))

Decision Tree Accuracy : 75.41%:


### Random Forest

In [61]:
rf_model = RandomForestClassifier(n_estimators=150, random_state=1)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy: {:.2%}:".format(rf_accuracy))

Random Forest Accuracy: 80.33%:


### SVM

In [68]:
svm_model = SVC(C=1.0, kernel='rbf', gamma='scale')
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test,svm_predictions)
print("SVM Acuuracy : {:.2%}:".format(svm_accuracy))


SVM Acuuracy : 67.21%:


### Logistic Regression

In [72]:
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)
logistic_predictions = logistic_model.predict(X_test)
logistic_accuracy = accuracy_score(y_test,logistic_predictions)
print("Logistic Regression Accuracy : {:.2%}:".format(logistic_accuracy))

Logistic Regression Accuracy : 80.33%:


### Naive Bayes

In [73]:
naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_train, y_train)
bayes_predictions = naive_bayes_model.predict(X_test)
bayes_accuracy = accuracy_score(y_test,bayes_predictions)
print("Naive Bayes Accuracy : {:.2%}:".format(bayes_accuracy))

Naive Bayes Accuracy : 81.97%:
